In [207]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable

In [208]:
train_data = []
train_labels = []
with open("optdigits.tra") as tra:
    for line in tra.readlines():
        row = [int(x) for x in line.split(",")]
        train_data.append(row[:-1])
        train_labels.append(row[64])

train_data = np.array(train_data)
train_labels = np.array(train_labels)

test_data = []
test_labels = []
with open("optdigits.tes") as tes:
    for line in tes.readlines():
        row = [int(x) for x in line.split(",")]
        test_data.append(row[:-1])
        test_labels.append(row[64])

test_data = np.array(test_data)
test_labels = np.array(test_labels)

train_data_scaled = train_data / 16.0
test_data_scaled = test_data / 16.0

In [209]:
print ("Train data size, dimensionX, dimensionY " + str(train_data.shape))
print (train_labels)
print ("Test data size, dimensionX, dimensionY " + str(test_data.shape))

Train data size, dimensionX, dimensionY (3823, 64)
[0 0 7 ... 6 6 7]
Test data size, dimensionX, dimensionY (1797, 64)


In [210]:
def train(train_data, train_labels,test_data, test_labels,loss,unit,layers,alpha,momentum,epochs):
    model = keras.Sequential([])
    
    for i in range(layers):
        model.add(keras.layers.Dense(128,activation=tf.nn.relu if unit.__eq__('relu') else tf.nn.tanh))
    
    model.add(keras.layers.Dense(10, activation=tf.nn.softmax))
    
    opt = optimizers.SGD(lr=alpha, decay=1e-6, momentum=momentum, nesterov=False)
    opt2 = optimizers.Adam(lr=alpha, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    model.compile(optimizer=opt2, 
                  loss=loss,
                  metrics=['accuracy'])
    model.fit(train_data, train_labels, epochs=epochs,verbose=0) #verbose=0 to suppress epoch prints
    test_loss, test_acc = model.evaluate(test_data, test_labels)
    
    return test_loss,test_acc


In [214]:
err_funcs = ['sparse_categorical_crossentropy','mean_squared_error']
units = ['relu','tanh']
alphas = [0.001,0.01,0.05,0.1]
layers = [1,3,7]
#momentums = [0.1,0.5,0.9
#scaling = [1.0,8.0,16.0]
#epochs = [5,10,20,50]
#accuracies = []


table = PrettyTable()
table.field_names = ["Err_Func", "Unit", "Learning R.", "Layers","epochs","Accuracy"]

count = 0
for err_func in err_funcs:
    for unit in units:
        for alpha in alphas:
            for layer in layers:
                _,acc = train(train_data_scaled, train_labels,test_data_scaled, test_labels,err_func,unit,layer,alpha,0.9,5)
                table.add_row([err_func,unit,alpha,layer,5,'{0:.2f} %'.format(acc * 100)])
                count += 1
                if count >= 5: break
            if count >= 5: break
        if count >= 5: break
    if count >= 5: break

print(table)


1797/1797 [==============================] - 2s 1ms/sample - loss: 0.1585 - acc: 0.9471
+---------------------------------+------+-------------+--------+--------+------------+
|             Err_Func            | Unit | Learning R. | Layers | epochs |  Accuracy  |
+---------------------------------+------+-------------+--------+--------+------------+
| sparse_categorical_crossentropy | relu |    0.001    |   1    |   5    | 0.94268227 |
| sparse_categorical_crossentropy | relu |    0.001    |   3    |   5    | 0.9571508  |
| sparse_categorical_crossentropy | relu |    0.001    |   7    |   5    | 0.9232053  |
| sparse_categorical_crossentropy | relu |     0.01    |   1    |   5    | 0.94045633 |
| sparse_categorical_crossentropy | relu |     0.01    |   3    |   5    | 0.94713414 |
+---------------------------------+------+-------------+--------+--------+------------+
